In [6]:
import os
import requests
import pandas as pd
from pymongo import MongoClient, ASCENDING
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# MongoDB Atlas connection setup
def get_mongo_client():
    mongo_uri = os.getenv("MONGO_URI")  # Get the MongoDB URI from .env file
    if not mongo_uri:
        raise ValueError("MONGO_URI is not set in the .env file")
    client = MongoClient(mongo_uri)
    return client

# Download CSV if not exists
def download_csv():
    if not os.path.exists("./FinalPermutations.csv"):
        s3_url = "https://coolmeal.s3.amazonaws.com/FinalPermutations.csv"
        print(f"File not found at ./FinalPermutations.csv. Downloading from S3...")
        response = requests.get(s3_url)

        if response.status_code == 200:
            with open("./FinalPermutations.csv", "wb") as f:
                f.write(response.content)
            print(
                f"File downloaded successfully from S3 and saved to ./FinalPermutations.csv."
            )
        else:
            raise FileNotFoundError(
                f"Failed to download file from {s3_url}. HTTP status code: {response.status_code}"
            )

# Add indexing and load data from CSV to MongoDB in chunks
def load_data_to_mongo_in_chunks():
    # Download the CSV file if not present
    download_csv()

    # Set chunk size (e.g., 1000 records per chunk)
    chunk_size = 1000
    total_inserted = 0  # Keep track of the total number of records inserted

    print(f"Reading data from ./FinalPermutations.csv in chunks of {chunk_size}...")

    # Set up the MongoDB connection
    client = get_mongo_client()
    db = client["cool-server"]
    collection = db["Meal_plans"]

    # Create an index on the 'index' field (optional but improves query performance)
    collection.create_index([('index', ASCENDING)], unique=True)
    print("Index created on the 'index' field.")

    # Read the CSV data in chunks
    for chunk_number, chunk in enumerate(pd.read_csv("./FinalPermutations.csv", chunksize=chunk_size)):
        print(f"Processing chunk {chunk_number + 1}...")

        # Add an 'index' column to each chunk starting from the last inserted index
        chunk['index'] = range(total_inserted, total_inserted + len(chunk))

        # Convert chunk to list of dictionaries
        records_to_insert = chunk.to_dict(orient='records')

        # Insert records into MongoDB
        collection.insert_many(records_to_insert)
        total_inserted += len(records_to_insert)  # Update total records inserted

        print(f"Inserted {len(records_to_insert)} records (Total inserted: {total_inserted})")

    print(f"All data inserted successfully. Total records inserted: {total_inserted}")

# Run the function to load data in chunks
load_data_to_mongo_in_chunks()


Reading data from ./FinalPermutations.csv in chunks of 1000...
Index created on the 'index' field.
Processing chunk 1...
Inserted 1000 records (Total inserted: 1000)
Processing chunk 2...
Inserted 1000 records (Total inserted: 2000)
Processing chunk 3...
Inserted 1000 records (Total inserted: 3000)
Processing chunk 4...
Inserted 1000 records (Total inserted: 4000)
Processing chunk 5...
Inserted 1000 records (Total inserted: 5000)
Processing chunk 6...
Inserted 1000 records (Total inserted: 6000)
Processing chunk 7...
Inserted 1000 records (Total inserted: 7000)
Processing chunk 8...
Inserted 1000 records (Total inserted: 8000)
Processing chunk 9...
Inserted 1000 records (Total inserted: 9000)
Processing chunk 10...
Inserted 1000 records (Total inserted: 10000)
Processing chunk 11...
Inserted 1000 records (Total inserted: 11000)
Processing chunk 12...
Inserted 1000 records (Total inserted: 12000)
Processing chunk 13...
Inserted 1000 records (Total inserted: 13000)
Processing chunk 14...

In [1]:
import os
import pandas as pd
from pymongo import MongoClient, ASCENDING
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# MongoDB Atlas connection setup
def get_mongo_client():
    mongo_uri = os.getenv("MONGO_URI")  # Get the MongoDB URI from .env file
    if not mongo_uri:
        raise ValueError("MONGO_URI is not set in the .env file")
    client = MongoClient(mongo_uri)
    return client

# Function to read CSV from a file path and upload to MongoDB
def upload_csv_to_mongo(file_path, db_name, collection_name):
    # Step 1: Read the CSV file using pandas
    try:
        data = pd.read_csv(file_path)
        print(f"CSV file '{file_path}' read successfully.")
    except Exception as e:
        raise Exception(f"Error reading CSV file: {str(e)}")
    
    # Step 2: Connect to MongoDB
    try:
        client = get_mongo_client()
        db = client[db_name]  # Access the specified database
        collection = db[collection_name]  # Access the specified collection
    except Exception as e:
        raise Exception(f"Error connecting to MongoDB: {str(e)}")
    
    # Step 3: Convert pandas DataFrame to a list of dictionaries (JSON format)
    records = data.to_dict(orient='records')

    # Step 4: Insert data into MongoDB collection
    try:
        result = collection.insert_many(records)
        print(f"Inserted {len(result.inserted_ids)} records into the '{collection_name}' collection.")
    except Exception as e:
        raise Exception(f"Error inserting records into MongoDB: {str(e)}")
    finally:
        client.close()

# Usage example (Replace with actual paths and MongoDB details)
upload_csv_to_mongo("../data-analytics/datasets/meals/DatasetMeals.csv","cool-server", "Meal")



CSV file '../data-analytics/datasets/meals/DatasetMeals.csv' read successfully.
Inserted 174 records into the 'Meal' collection.


In [2]:
# Usage example (Replace with actual paths and MongoDB details)
upload_csv_to_mongo("../data-analytics/datasets/meals/FinalMeals.csv","cool-server", "Final_Meals")

CSV file '../data-analytics/datasets/meals/FinalMeals.csv' read successfully.
Inserted 10468 records into the 'Final_Meals' collection.


Test adding new meal p;lan

In [7]:
import pandas as pd
import os

# The function to add meal plan to CSV
def add_meal_plan_to_csv(meal_plan_dict):
    # Define the path to the CSV file
    csv_file_path = "./FinalPermutations.csv"

    # Check if the CSV file exists
    if os.path.exists(csv_file_path):
        # Load the existing data into a DataFrame
        df = pd.read_csv(csv_file_path)

        # Create a DataFrame for the new meal plan
        new_meal_plan_df = pd.DataFrame([meal_plan_dict])

        # Concatenate the new meal plan to the existing DataFrame
        df = pd.concat([df, new_meal_plan_df], ignore_index=True)

        # Save the updated DataFrame back to the CSV file
        df.to_csv(csv_file_path, index=False)
    else:
        # If the file does not exist, create a new DataFrame and save it
        df = pd.DataFrame([meal_plan_dict])
        df.to_csv(csv_file_path, index=False)

# Test function
def test_add_meal_plan_to_csv():
    # Sample meal plan data
    meal_plan_dict = {
    
    "Breakfast": "Red Rice with Fish Curry, Tapioca Chips, Batu Moju",
    "Lunch": "Red Rice with Banana Blossom Curry, Green Gram Curry, Fenugreek Curry",
    "Dinner": "Vegetable Kottu with Devilled Fish",
    "Price": 792.3,
    "Total Energy(Kcal)": 1482.37,
    "Total Protein(g)": 73.5357,
    "Total Total fat(g)": 83.1566,
    "Total Carbohydrates(g)": 175.9371,
    "Total Magnesium(mg)": 478.5,
    "Total Sodium(mg)": 128.367,
    "Total Potassium(mg)": 3346.98,
    "Total Saturated Fatty Acids(mg)": 37105.9815,
    "Total Monounsaturated Fatty Acids(mg)": 6314.2741,
    "Total Polyunsaturated Fatty Acids(mg)": 5152.7615,
    "Total Free sugar(g)": 11.7442,
    "Total Starch(g)": 151.1037,
    "index": 0
}


    # Clear any existing CSV file for testing
    csv_file_path = "./FinalPermutations.csv"
    
    # Call the function to add meal plan to CSV
    add_meal_plan_to_csv(meal_plan_dict)

    # Read the CSV file to verify the contents
    df = pd.read_csv(csv_file_path)

    # Check if the meal plan was added correctly
    print(df.shape)

    print("Test passed! Meal plan was added to CSV successfully.")

# Run the test
test_add_meal_plan_to_csv()


(200001, 17)
Test passed! Meal plan was added to CSV successfully.


In [8]:
import pandas as pd
csv_file_path = "./FinalPermutations.csv"
df = pd.read_csv(csv_file_path)
df.tail()

,Breakfast,Lunch,Dinner,Price,Total Energy(Kcal),Total Protein(g),Total Total fat(g),Total Carbohydrates(g),Total Magnesium(mg),Total Sodium(mg),Total Potassium(mg),Total Saturated Fatty Acids(mg),Total Monounsaturated Fatty Acids(mg),Total Polyunsaturated Fatty Acids(mg),Total Free sugar(g),Total Starch(g),index
199996,"Red Rice with Parippu, Tomato Curry, Vegetable...","Chicken Fried Rice with Ala Badun, Cashew Curry",Coconut Roti with Fish Ambul Thiyal,1082.542000,1775.250,70.30400,116.97850,192.51750,500.0985,138.2080,3251.318,46516.95900,21770.17700,8176.2990,11.33950,165.6770,199996
199997,Hoppers,"Red Rice with Chicken Curry, Beetroot curry, K...","Vegetable Fried Rice with Wambatu Moju, Cashew...",611.507857,1755.605,53.40785,117.71200,223.42295,405.8020,121.7301,2307.994,52973.73200,20356.11515,6584.9755,18.49190,172.2911,199997
199998,Gotu Kola Kanda,"Red Rice with Pineapple Curry, Beetroot curry,...","Egg Fried Rice with Ala Badun, Fish Cutlets",483.850000,1384.290,43.96545,91.19495,164.94830,337.0390,213.4383,2599.690,35868.61975,7705.21390,3619.7880,14.26885,140.1092,199998
199999,"Red Rice with Seer Fish Curry, Radish Sambol, ...","Red Rice with Prawn Curry, Vegetable Cutlets, ...","Red Rice with Potato Curry, Carrot Curry, Gree...",671.300000,1490.120,74.83070,148.05180,163.78600,517.3570,279.4952,4262.450,95988.91800,11371.37850,5383.5840,20.44600,126.2131,199999
200000,"Red Rice with Fish Curry, Tapioca Chips, Batu ...","Red Rice with Banana Blossom Curry, Green Gram...",Vegetable Kottu with Devilled Fish,792.300000,1482.370,73.53570,83.15660,175.93710,478.5000,128.3670,3346.980,37105.98150,6314.27410,5152.7615,11.74420,151.1037,0


In [9]:
def convert_meal_plan_dict(original_dict):
    # Conversion logic
    converted_dict = {
        "Breakfast": original_dict["Breakfast"],
        "Lunch": original_dict["Lunch"],
        "Dinner": original_dict["Dinner"],
        "Price": original_dict["Price"],
        "Total Energy(Kcal)": original_dict["Total_Energy_Kcal"],
        "Total Protein(g)": original_dict["Total_Protein_g"],
        "Total Total fat(g)": original_dict["Total_Fat_g"],
        "Total Carbohydrates(g)": original_dict["Total_Carbohydrates_g"],
        "Total Magnesium(mg)": original_dict["Total_Magnesium_mg"],
        "Total Sodium(mg)": original_dict["Total_Sodium_mg"],
        "Total Potassium(mg)": original_dict["Total_Potassium_mg"],
        "Total Saturated Fatty Acids(mg)": original_dict["Total_Saturated_Fatty_Acids_mg"],
        "Total Monounsaturated Fatty Acids(mg)": original_dict["Total_Monounsaturated_Fatty_Acids_mg"],
        "Total Polyunsaturated Fatty Acids(mg)": original_dict["Total_Polyunsaturated_Fatty_Acids_mg"],
        "Total Free sugar(g)": original_dict["Total_Free_Sugar_g"],
        "Total Starch(g)": original_dict["Total_Starch_g"],
        "index": original_dict["index"]
    }
    return converted_dict

# Example usage
original_dict = {
    'Breakfast': 'Red Rice with Fish Curry, Tapioca Chips, Batu Moju',
    'Lunch': 'Red Rice with Banana Blossom Curry, Green Gram Curry, Fenugreek Curry',
    'Dinner': 'Vegetable Kottu with Devilled Fish',
    'Price': 792.3,
    'Total_Energy_Kcal': 1482.37,
    'Total_Protein_g': 73.5357,
    'Total_Fat_g': 83.1566,
    'Total_Carbohydrates_g': 175.9371,
    'Total_Magnesium_mg': 478.5,
    'Total_Sodium_mg': 128.367,
    'Total_Potassium_mg': 3346.98,
    'Total_Saturated_Fatty_Acids_mg': 37105.9815,
    'Total_Monounsaturated_Fatty_Acids_mg': 6314.2741,
    'Total_Polyunsaturated_Fatty_Acids_mg': 5152.7615,
    'Total_Free_Sugar_g': 11.7442,
    'Total_Starch_g': 151.1037,
    'index': 232323
}

converted_dict = convert_meal_plan_dict(original_dict)
print(converted_dict)

{'Breakfast': 'Red Rice with Fish Curry, Tapioca Chips, Batu Moju', 'Lunch': 'Red Rice with Banana Blossom Curry, Green Gram Curry, Fenugreek Curry', 'Dinner': 'Vegetable Kottu with Devilled Fish', 'Price': 792.3, 'Total Energy(Kcal)': 1482.37, 'Total Protein(g)': 73.5357, 'Total Total fat(g)': 83.1566, 'Total Carbohydrates(g)': 175.9371, 'Total Magnesium(mg)': 478.5, 'Total Sodium(mg)': 128.367, 'Total Potassium(mg)': 3346.98, 'Total Saturated Fatty Acids(mg)': 37105.9815, 'Total Monounsaturated Fatty Acids(mg)': 6314.2741, 'Total Polyunsaturated Fatty Acids(mg)': 5152.7615, 'Total Free sugar(g)': 11.7442, 'Total Starch(g)': 151.1037, 'index': 232323}
